# RNN用于图像分类
RNN 一般是用做时间序列的数据，在这里，也可以将图像数据看成时间序列的，将每行像素看做输入的数据
对于minist数据集，训练图片为手写数字图片，标签为相应的整型数字,需要注意的是，使用crossEntropy进行分类的时候，函数会自动将标签对应的数字标签转换为onehot形式进行处理
## Appeals
One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task<br/>
## Features
the key to LSTMS is the cell state. The cell state is kind of like a conveyor belt. It runs straght down the entire chain, with only some minor linear interactions. It's very easy for information to just flow along it unchanged. The LSTM dows have the ability to remove or add information to the cell state, carefully regulated by structures called gates. 

In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
EPOCH=1
BATCH_SIZE=64
TIME_STEP=28
INPUT_SIZE=28
LR=0.01
DOWNLOAD_MNIST=False

In [3]:
train_data=dsets.MNIST(root='./mnist',train=True,transform=transforms.ToTensor(),download=False)
train_loader=torch.utils.data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)

In [4]:
test_data=dsets.MNIST(root='./mnist',train=False,transform=transforms.ToTensor())

In [5]:
test_x=test_data.test_data[:2000].type(torch.FloatTensor)/255
test_y=test_data.test_labels[:2000].type(torch.FloatTensor)

In [6]:
class RNN(nn.Module):
    def __init___(self):
        super(RNN,self).__init__()
        self.rnn=nn.LSTM(input_size=INPUT_SIZE,
                        hidden_size=64,
                        num_layers=1,
                        batch_first=True,)
        self.out=nn.Linear(64,10)
        
    def forward(self,x):
        r_out,(h_n,h_c)=self.rnn(x,None)
        out=self.out(r_out[:,-1,:])#(batch,time_step,input)
        return out
    
    


In [7]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        
        self.rnn=nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out=nn.Linear(64,10)
        
    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [8]:
rnn=RNN()
print(rnn,)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [24]:
optimizer=torch.optim.Adam(rnn.parameters(),lr=LR)
loss_func=nn.CrossEntropyLoss()
for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader):
        b_x=x.view(-1,28,28)
        b_y=y
        output=rnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step%50==0:
            test_output=rnn(test_x)
            pred_y=torch.max(test_output,1)[1].data.numpy().squeeze()
            print (test_y.size()[0])
            accuracy=sum(pred_y==test_y)/test_y.size()[0]
            print ('Epoch:',epoch,'|train loss:%.4f'%loss.data[0],'| test accuracy:%.4f'%accuracy)

2000


RuntimeError: value cannot be converted to type uint8_t without overflow: 2000